In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').appName('pyspark_fresh').getOrCreate()

In [3]:
from pyspark.sql import Row

In [4]:
import datetime
data= [{
    "id":1,
    "firstName":"Terry",
    "lastName":"Medhurst",
    "email":"atuny0@sohu.com",
    "phone":Row(phone_number = "+63 791 675 8914",mobile_number = "+63 791 675 8945"),
    "courses" : [1,2],
    "is_customer" : True,
    "amount_paid" : 10200,
    "customer_from" : datetime.date(2022,3,10),
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
},
{
    "id":2,
    "firstName":"Sheldon",
    "lastName":"Quigley",
    "email":"hbingley1@plala.or.jp",
    "phone":Row(phone_number="+7 813 117",mobile_number="+7 813 119"),
    "courses" : [3],
    "is_customer" : True,
    "amount_paid" : 11000,
    "customer_from" : datetime.date(2022,5,16),
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
},
{
    "id":3,
    "firstName":"Terrill",
    "lastName":"Hills",
    "email":"rshawe2@51.la",
    "phone":Row(phone_number="+63 739 292 7942",mobile_number="+63 739 292 7943"),
    "courses" : [2,4],
    "is_customer" : True,
    "amount_paid" : 24000,
    "customer_from" : datetime.date(2022,4,11),
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
},
{
    "id":4,
    "firstName":"Miles",
    "lastName":"Cummerata",
    "email":"yraigatt3@nature.com",
    "phone":Row(phone_number=None,mobile_number=None),
    "courses" : [],
    "is_customer" : False,
    "amount_paid" : None,
    "customer_from" : None,
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
},
{
    "id":5,
    "firstName":"Mavis",
    "lastName":"Schultz",
    "email":"kmeus4@upenn.edu",
    "phone":Row(phone_number="+372 285 771 1911",mobile_number=None),
    "courses" : [],
    "is_customer" : False,
    "amount_paid" : None,
    "customer_from" : None,
    "last_update_ts" : datetime.datetime(2023,10,20, 15,0,45)
}]

In [5]:
import pandas as pd

In [6]:
df = spark.createDataFrame(pd.DataFrame(data))

In [8]:
%md
.Concatenate firstName and lastName to generate fullName

SyntaxError: invalid syntax (2021240318.py, line 2)

In [9]:
from pyspark.sql.functions import col,lit,concat

In [12]:
fullNameCol = concat(col('firstName'),lit(', '),col('lastName'))

In [13]:
fullNameCol

Column<'concat(firstName, , , lastName)'>

In [14]:
fullNameAlias = fullNameCol.alias('fullName')

In [15]:
df.select('id',fullNameAlias).show()

+---+----------------+
| id|        fullName|
+---+----------------+
|  1| Terry, Medhurst|
|  2|Sheldon, Quigley|
|  3|  Terrill, Hills|
|  4|Miles, Cummerata|
|  5|  Mavis, Schultz|
+---+----------------+



In [16]:
df.select('id','customer_from').show()

+---+-------------+
| id|customer_from|
+---+-------------+
|  1|   2022-03-10|
|  2|   2022-05-16|
|  3|   2022-04-11|
|  4|         null|
|  5|         null|
+---+-------------+



In [17]:
from pyspark.sql.functions import date_format

In [24]:
date_format('customer_from','yyyyMMdd')

Column<'date_format(customer_from, yyyyMMdd)'>

In [25]:
date_format('customer_from','yyyyMMdd').cast('int')

Column<'CAST(date_format(customer_from, yyyyMMdd) AS INT)'>

In [26]:
coustomer_from_alias = date_format('customer_from','yyyyMMdd').cast('int').alias('customer_from')

In [29]:
df.select('id',fullNameAlias,coustomer_from_alias).show()

+---+----------------+-------------+
| id|        fullName|customer_from|
+---+----------------+-------------+
|  1| Terry, Medhurst|     20220310|
|  2|Sheldon, Quigley|     20220516|
|  3|  Terrill, Hills|     20220411|
|  4|Miles, Cummerata|         null|
|  5|  Mavis, Schultz|         null|
+---+----------------+-------------+



In [30]:
df.select('id',fullNameAlias,coustomer_from_alias).dtypes

[('id', 'bigint'), ('fullName', 'string'), ('customer_from', 'int')]

In [35]:
df.createOrReplaceTempView('udf')

In [36]:
spark.sql(
"""
SELECT id, (amount_paid+25) AS amount_paid FROM udf
""").show()

+---+-----------+
| id|amount_paid|
+---+-----------+
|  1|    10225.0|
|  2|    11025.0|
|  3|    24025.0|
|  4|        NaN|
|  5|        NaN|
+---+-----------+



In [37]:
df.selectExpr('id','(amount_paid+25) AS amount_paid').show()

+---+-----------+
| id|amount_paid|
+---+-----------+
|  1|    10225.0|
|  2|    11025.0|
|  3|    24025.0|
|  4|        NaN|
|  5|        NaN|
+---+-----------+



In [38]:
df.select('id','amount_paid'+'25').show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `amount_paid25` cannot be resolved. Did you mean one of the following? [`amount_paid`, `email`, `courses`, `firstName`, `id`].;
'Project [id#0L, 'amount_paid25]
+- LogicalRDD [id#0L, firstName#1, lastName#2, email#3, phone#4, courses#5, is_customer#6, amount_paid#7, customer_from#8, last_update_ts#9], false


In [39]:
lit(25)

Column<'25'>

In [40]:
'amount_paid'+lit(25)

Column<'(25 + amount_paid)'>

In [44]:
df.select('id','amount_paid'+lit(25.0),lit(50.0)+lit(25.0)).show()

+---+--------------------+-------------+
| id|(25.0 + amount_paid)|(50.0 + 25.0)|
+---+--------------------+-------------+
|  1|                null|         75.0|
|  2|                null|         75.0|
|  3|                null|         75.0|
|  4|                null|         75.0|
|  5|                null|         75.0|
+---+--------------------+-------------+



In [45]:
df.select('id',col('amount_paid')+lit(25.0)).show()

+---+--------------------+
| id|(amount_paid + 25.0)|
+---+--------------------+
|  1|             10225.0|
|  2|             11025.0|
|  3|             24025.0|
|  4|                 NaN|
|  5|                 NaN|
+---+--------------------+

